In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## General Port Parameters

In [2]:
class general_parameters():
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.year              = 2018   # current year within simulation
        self.start_year        = 2018   # start year of simulation
        self.timestep          = self.year - self.start_year
        self.operational_hours = 8760   # operational hours per year
       
parameters = general_parameters()

# Scenario Generator

In [3]:
from  terminal_optimization.forecast_package import maize, soybean, wheat, handysize, handymax, panamax

In [4]:
# Generating a single demand forecast

window         = 20         # looking 20 years ahead
trend_type     = 'linear'   # linear, constant or random

demand_maize   = 1000000    # demand at t=0
growth_maize   = 100000     # year on year absolute growth of demand        - input for linear method
rate_maize     = 1.01       # year on year growth rate of demand (% points) - input for constant method and random method
mu_maize       = 0          # avg bonus rate added to base rate (% points)  - input for random method
sigma_maize    = 0          # standard deviation of bonus rate (% points)   - input for random method

demand_soybean = 0
growth_soybean = 0
rate_soybean   = 0
mu_soybean     = 0
sigma_soybean  = 0

demand_wheat   = 0
growth_wheat   = 0
rate_wheat     = 0
mu_wheat       = 0
sigma_wheat    = 0

maize.generate_forecast  (trend_type, parameters.year, window, demand_maize,   growth_maize  , rate_maize,   mu_maize,   sigma_maize)
soybean.generate_forecast(trend_type, parameters.year, window, demand_soybean, growth_soybean, rate_soybean, mu_soybean, sigma_soybean)
wheat.generate_forecast  (trend_type, parameters.year, window, demand_wheat,   growth_wheat  , rate_wheat,   mu_wheat,   sigma_wheat)

# Port Model

In [5]:
import terminal_optimization.investment_decision_package as module

True!!!


In [6]:
# exporting objects from notebook to investment_decision_package 
module.parameters = parameters
module.maize      = maize
module.soybean    = soybean
module.wheat      = wheat
module.handysize  = handysize
module.handymax   = handymax
module.panamax    = panamax
module.import_notebook_parameters()
module.import_notebook_classes()

In [7]:
# exporting objects from investment_decision_package to infrastructure package 
module.export_infrastructure_package()

In [8]:
#module.number_of_berths_calc()
#module.quay_wall_length_calc()
#module.berths[0].__dict__

In [9]:
#module.berths[0].quantity
module.berth_invest_decision()
#module.quay.length

True